In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

### Read in data

In [2]:
# # load dataset and specify predictors/target
# df = pd.read_csv('../data/alldata.csv', index_col='Unnamed: 0')
# df = df[df.stop_outcome.isin([1,4])]
# def change_1_0(x):
#     if x == 4: return 0
#     else: return 1
# df.stop_outcome = df.stop_outcome.apply(lambda x: change_1_0(x))
# predictors = df.drop(['stop_outcome'], axis = 1)
# target = df.stop_outcome
# target.value_counts()/len(target)

In [3]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y = True)

In [4]:
# split data
# X_train , X_test , y_train , y_test = train_test_split (predictors, target , test_size=0.2, random_state = 9)
X_train , X_test , y_train , y_test = train_test_split (X, y , test_size=0.2, random_state = 9)
# normalize data
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [7]:
# y_train = y_train.values
# y_test = y_test.values

### Model

In [5]:
class LogisticRegression():
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y, batch_size):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in tqdm(range(self.num_iter)):
            rand_indices = np.random.choice(X.shape[0], size = batch_size, replace = True)
            X_ = X[rand_indices, :]
            y_ = y[rand_indices]

            z = np.dot(X_, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X_.T, (h - y_)) / y.size
            self.theta -= self.lr * gradient
            
            if self.verbose == True and i % 1000 == 0:
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y)} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold

In [7]:
model = LogisticRegression(lr=0.01, num_iter=100000, verbose=True)
% time model.fit(X_train, y_train, batch_size=64)
preds = model.predict(X_train, threshold = 0.5)
# accuracy
accuracy_score(y_train, preds)


CPU times: user 6.19 s, sys: 86.3 ms, total: 6.28 s
Wall time: 6.52 s


0.9472527472527472

In [22]:
class DPLogisticRegression():
    def __init__(self, lr=0.01, num_iter=100000, 
                 fit_intercept=True, verbose=False,
                 clipping_param = 5, sigma = 2,
                 delta = 1e-6,
                sample_with_replacement_SGD = True):
        
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
        self.replacement = sample_with_replacement_SGD
        
        # Privacy parameters
        self.clipping_param = clipping_param
        self.sigma = sigma
        self.delta = delta
        self.epsilon = None
        
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    def __get_min_epsilon(self, batch_size, X):
        return self.clipping_param/self.sigma*batch_size/X.shape[0]*np.sqrt(X.shape[0]//batch_size * np.log(1/self.delta))
    
    def fit(self, X, y, batch_size):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in tqdm(range(self.num_iter)):
            # fix this to not sample same row twice - or make an option
            rand_indices = np.random.choice(X.shape[0], size = batch_size, replace = self.replacement)
            X_ = X[rand_indices, :]
            y_ = y[rand_indices]

            z = np.dot(X_, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X_.T, (h - y_)) / y_.size
            
            # clip the gradient using microbatch concept - fix this by calculating gradient example by example
            gradient_clipped = gradient/max(1, np.linalg.norm(gradient))
            gradient_noisy = gradient_clipped + 1/y_.size * (np.random.normal(loc = 0.0, scale = self.sigma * self.clipping_param))
            
#             print('thetas: \t', self.theta)
            self.theta -= self.lr * gradient_noisy
            
            if self.verbose == True and i % 1000 == 0:
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y)} \t')
        
        self.epsilon = self.__get_min_epsilon(batch_size, X)
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold

In [26]:
model = DPLogisticRegression(lr=0.001, num_iter=100000, verbose=False, fit_intercept = True, 
                            clipping_param=1, sigma=2, delta=1e-2)
% time model.fit(X_train, y_train, batch_size=1)
preds = model.predict(X_train, threshold = 0.5)
# accuracy
accuracy_score(y_train, preds)


CPU times: user 7.44 s, sys: 129 ms, total: 7.57 s
Wall time: 7.73 s


0.9252747252747253

In [24]:
test_preds = model.predict(X_test, threshold = 0.5)
# accuracy
(test_preds == y_test).mean()

0.6491228070175439

In [25]:
model.epsilon

0.0010060443904727954